In [1]:
import numpy as np
import pandas as pd

import requests
import re
import os
from  bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep
import random

In [2]:
browser = webdriver.Chrome()

browser.get("https://nba.com/games")
browser.maximize_window()

# cookies
sleep(1)
browser.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()

# Ir a la página de estadísticas
sleep(3.2)
browser.find_element(by=By.XPATH, value='//*[@id="nav-ul"]/li[5]/a').click()

# Ir a la opción de SEASON LEADERS
sleep(1.4)
browser.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/div[2]/div[3]/div/div[1]/section[1]/div/div[2]/button[2]').click()


In [3]:
html_season_leaders = browser.page_source
soup = BeautifulSoup(html_season_leaders, "html.parser")
url_estadisticas_lideres = [a['href'] for a in soup.find_all('a', class_='Anchor_anchor__cSc3P LeaderBoardCard_lbcLink__GZTl1 Link_styled__okbXW')]
print(url_estadisticas_lideres)

['/stats/leaders/', '/stats/leaders/?StatCategory=REB', '/stats/leaders/?StatCategory=AST', '/stats/leaders/?StatCategory=BLK', '/stats/leaders/?StatCategory=STL', '/stats/leaders/?StatCategory=FG_PCT&PerMode=Totals', '/stats/leaders/?StatCategory=FG3M&PerMode=Totals', '/stats/leaders/?StatCategory=FG3_PCT&PerMode=Totals', '/stats/players/traditional/?sort=NBA_FANTASY_PTS&dir=-1', '/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1', '/stats/players/traditional/?sort=REB&dir=-1&PerMode=Totals', '/stats/players/traditional/?sort=AST&dir=-1&PerMode=Totals', '/stats/players/traditional/?sort=BLK&dir=-1&PerMode=Totals', '/stats/players/traditional/?sort=STL&dir=-1&PerMode=Totals', '/stats/players/traditional/?sort=FG3M&dir=-1&PerMode=Totals', '/stats/players/boxscores-traditional/?sort=FG3M&dir=1', '/stats/players/boxscores-traditional/?sort=PTS&dir=1', '/stats/teams/scoring/?sort=PCT_PTS_3PT&dir=-1', '/stats/players/catch-shoot/?sort=CATCH_SHOOT_FGA&dir=1']


In [4]:
dominio_base = "https://www.nba.com"
urls_estadisticas_lideres = [dominio_base + url for url in url_estadisticas_lideres]

# Imprimir los enlaces absolutos para verificar
for url in urls_estadisticas_lideres:
    print(url)

https://www.nba.com/stats/leaders/
https://www.nba.com/stats/leaders/?StatCategory=REB
https://www.nba.com/stats/leaders/?StatCategory=AST
https://www.nba.com/stats/leaders/?StatCategory=BLK
https://www.nba.com/stats/leaders/?StatCategory=STL
https://www.nba.com/stats/leaders/?StatCategory=FG_PCT&PerMode=Totals
https://www.nba.com/stats/leaders/?StatCategory=FG3M&PerMode=Totals
https://www.nba.com/stats/leaders/?StatCategory=FG3_PCT&PerMode=Totals
https://www.nba.com/stats/players/traditional/?sort=NBA_FANTASY_PTS&dir=-1
https://www.nba.com/stats/players/traditional/?PerMode=Totals&sort=PTS&dir=-1
https://www.nba.com/stats/players/traditional/?sort=REB&dir=-1&PerMode=Totals
https://www.nba.com/stats/players/traditional/?sort=AST&dir=-1&PerMode=Totals
https://www.nba.com/stats/players/traditional/?sort=BLK&dir=-1&PerMode=Totals
https://www.nba.com/stats/players/traditional/?sort=STL&dir=-1&PerMode=Totals
https://www.nba.com/stats/players/traditional/?sort=FG3M&dir=-1&PerMode=Totals
http

In [5]:
# Vamos a hacer primero el ejemplo para sacar los datos del primer enlace, y luego y ya montamos el bucle

primer_url = urls_estadisticas_lideres[0]
browser.get(primer_url)

# Vamos a darle que nos de el total de todas las estadísticas
sleep(3)
browser.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[3]/label/div/select/option[1]').click()

# Vamos a decirle que nos ponga a todos los jugadores dándole al ALL
sleep(3.3)
browser.find_element(by=By.XPATH, value='//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]').click()
                                         

In [6]:
# Una vez estamos ya dentro de la página, vamos a extraer el nombre de las columnas y creamos listas
# Lo vamos a hacer en una función para poder ejecutarla posteriormente en otras tablas similares

html_tabla = browser.page_source

def nombre_columnas_estadisticas(html_tabla):
    soup = BeautifulSoup(html_tabla, "html.parser")

    encabezados_tabla = soup.find('tr', class_='Crom_headers__mzI_m')

    if encabezados_tabla:
        nombres_columnas = [th.get_text(strip=True) for th in encabezados_tabla.find_all('th')]
        return nombres_columnas
    else:
         return []
    
nombres_de_columnas = nombre_columnas_estadisticas(html_tabla)
print(nombres_de_columnas)

['#', 'Player', 'TEAM', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'EFF', 'AST/TOV', 'STL/TOV']


In [7]:
print(len(nombres_de_columnas))

26


In [8]:
# Ahora vamos a crear una función para descargar los datos de cada jugador. 
# En la primera columna, en vez de descargar la posición vamos a descargar la ID del jugador que sale del enlace para identificarlo
# Lo mismo haremos con el equipo, haremos la ID del equipo

html_fila = browser.page_source

def extraer_datos_jugadores(html_fila):
    soup = BeautifulSoup(html_fila, "html.parser")
    columnas = soup.find_all('td')

    enlace_jugador = columnas[1].find('a', href=True)
    id_jugador = re.search(r'/stats/player/(\d+)/', enlace_jugador['href']).group(1) if enlace_jugador else None

    nombre_jugador = columnas[1].get_text(strip=True) if columnas[1] else None


    enlace_equipo = columnas[2].find('a', href=True)
    id_equipo = re.search(r'/stats/team/(\d+)/', enlace_equipo['href']).group(1) if enlace_equipo else None

    otros_datos = [columna.get_text(strip=True) for columna in columnas[3:]]

    datos = [id_jugador, nombre_jugador, id_equipo] + otros_datos

    return datos




In [9]:
nombres_columnas = nombre_columnas_estadisticas(html_tabla)

datos_jugadores = []

soup = BeautifulSoup(html_tabla, "html.parser")
filas = soup.find_all('tr')[1:]  # Esto es para excluir la fila de encabezado

for fila in filas:
    datos_fila = str(fila)  
    datos_jugador = extraer_datos_jugadores(datos_fila)
    datos_jugadores.append(datos_jugador)


df_estadisticas_puntos_jugadores = pd.DataFrame(datos_jugadores, columns=nombres_columnas)


print(df_estadisticas_puntos_jugadores)


           #                   Player        TEAM  GP   MIN   PTS  FGM  FGA  \
0     203507    Giannis Antetokounmpo  1610612749  36  1268  1132  420  692   
1    1629029              Luka Doncic  1610612742  33  1215  1111  379  779   
2    1628983  Shai Gilgeous-Alexander  1610612760  34  1184  1071  384  699   
3     203999             Nikola Jokic  1610612743  37  1235   942  366  643   
4    1628973            Jalen Brunson  1610612752  36  1291   937  331  701   
..       ...                      ...         ...  ..   ...   ...  ...  ...   
516  1641931            Onuralp Bitim  1610612741   1     3     0    0    0   
517  1641741         Ricky Council IV  1610612755   1     6     0    0    1   
518  1631199           Ron Harper Jr.  1610612761   1     4     0    0    0   
519  1627853         Ryan Arcidiacono  1610612752  13    26     0    0    4   
520  1630586             Usman Garuba  1610612744   2     6     0    0    1   

      FG%  3PM  ... DREB  REB  AST STL BLK  TOV   P

In [11]:
# Hacemos el guardado que estamos ya realizando para si existse el archivo, que no se sobreescriba

def guardar_excel_con_numeracion(df, nombre_base):
    directorio = os.path.join(os.getcwd(), "excels")

    if not os.path.exists(directorio):
        os.makedirs(directorio)

    ruta_base = os.path.join(directorio, nombre_base)

    contador = 0
    ruta_final = f"{ruta_base}.xlsx"

    while os.path.exists(ruta_final):
        contador += 1
        ruta_final = f"{ruta_base}_{contador}.xlsx"

    df.to_excel(ruta_final, index=False, engine='openpyxl')
    print(f"Archivo guardado como: {ruta_final}")

guardar_excel_con_numeracion(df_estadisticas_puntos_jugadores, "estadisticas_puntos_jugadores")


Archivo guardado como: c:\Users\34670\Desktop\python\Hack a boss\proyecto_1\excels\estadisticas_puntos_jugadores_2.xlsx
